# Quinto Projeto: Geração de Código

Após a conclusão da análise semântica, podemos percorrer a AST decorada para gerar código intermediário. Utilizaremos como representação intermediária a linguagem C, usando-a como uma espécie de linguagem de máquina de baixo nível.

À primeira vista, isso pode parecer um tanto sem sentido, mas na verdade faz sentido por vários motivos. Primeiro, os compiladores C são padronizados e onipresentes – estando disponíveis em qualquer lugar. Segundo, C tem recursos tão mínimos que você pode usá-lo em um nível baixo sem ter que se preocupar com efeitos colaterais ocultos (ou seja, coleta de lixo, etc). Você também pode abusar de todos os tipos de maneiras horríveis de realizar atos não naturais de programação (útil ao criar uma nova linguagem). Por fim, para fazer seu compilador realmente “funcionar”, pode haver certos elementos de tempo de execução que são bastante difíceis de implementar ou depurar diretamente com instruções de máquina. Portanto, pode ser mais fácil implementar coisas em C como primeiro passo.

Este caderno está dividido em duas partes. Na primeira, são mostrados alguns exemplos de transformação de um programa [uChuck](https://colab.research.google.com/drive/1GiV8weG5lzVvA7z970EiE8bUMj3DAg0x?usp=sharing) em código C. A segunda parte fornece um conjunto de diretrizes, bem como alguns trechos de código que devem servir como guia para a geração do código C. Com algumas exceções, como o nome de algumas classes, métodos e atributos que interagem com os outros componentes do compilador e que são explicados nesta etapa, você tem liberdade para modificar o código fornecido.

## Parte I: Representação Intermediária

Nesta etapa do projeto, você percorrerá a AST decorada para gerar um código em linguagem C. Nosso objetivo é produzir código C de baixo nível onde você **SOMENTE** possa usar os seguintes recursos de C:

1. Você só pode declarar variáveis não inicializadas. Por exemplo:

```c
        int x;
        float y;
        int z = 2 + 3;     // NO!!!!!!!!
```

Todas as variáveis devem ser declaradas antes do uso, no topo do arquivo ou função. Lembre-se: elas não são inicializadas.

2. Você só pode realizar **UMA** operação por vez. As operações só podem envolver constantes e variáveis. O resultado de uma operação deve sempre ser armazenado imediatamente em uma variável. Por exemplo, para calcular 2 + 3 * 4, você pode fazer o seguinte:

```c
        int _t1;      /* Temporary variables */
        int _t2;

        _t1 = 3 * 4;
        _t2 = 2 + _t1;
```         

3. As únicas construções de fluxo de controle permitidas são os dois comandos a seguir:

```c
       goto Label;
       if (_variable) goto Label;

       Label:
          /* ... code ... */
```

Não, você não tem "else". E você também não tem blocos de código entre colchetes { }.

4. Você pode imprimir coisas com `printf()`.

É isso, esses são os únicos recursos de C que você pode usar. Para ajudá-lo, aqui está um pequeno fragmento de código do [uChuck](https://colab.research.google.com/drive/1GiV8weG5lzVvA7z970EiE8bUMj3DAg0x?usp=sharing) que executa um laço:

```
   /* Sample uChuck Code */
   /* Print the first 10 factorials */

   1 => int result;
   1 => int n;

   while( n < 10 )
   {
	     result * n => result;
	     <<< result >>>;
	     n + 1 => n;
   }
```

Esta é a aparência do código C correspondente (pode variar desde que você observe as regras gerais acima):

```c
   #include <stdio.h>
   #include <stdlib.h>
   #include <string.h>

   int result;
   int n;

   int main() {
       int _t1;
       int _t2;
       int _t3;

       result = 1;
       n = 1;
   L1:
       _t1 = n < 10;
       if (_t1) goto L2;
       goto L3;
   L2:
       _t2 = result * n;
       result = _t2;
       printf("%i\n", result);
       _t3 = n + 1;
       n = _t3;
       goto L1;
   L3:
       return 0;
   }
```

Uma coisa a ter em mente... o objetivo **NÃO** é produzir código para ser lido por humanos. Em vez disso, é para produzir código mínimo e que possa ser fundamentado. Na verdade, há muito pouco acontecendo no código acima. Possui cálculos e gotos. É basicamente isso. Não há nenhuma mágica invisível de alto nível acontecendo. O que você vê é o que é.


## Parte II: Geração de Código

Para gerar o código C, você precisa percorrer a AST. Para isso, use a classe `NodeVisitor` fornecida anteriormente. Um trecho da classe `CodeGenerator` é mostrado abaixo. Você pode adaptar este código como desejar, mas preservando o nome da classe e o método `show`.

```python
# Encapsulation of a C function
class Function:
    
    def __init__(self, funcname, argdefns, rettype):
        self.funcname = funcname
        self.argdefns = argdefns
        self.rettype = rettype
        self.locals = []
        self.statements = []
        
    def __str__(self):
        return (f'{self.rettype} {self.funcname}(' + ', '.join(self.argdefns) + ") {\n" +
                ('\n'.join(self.locals) + "\n\n" if len(self.locals) else "") +
                ('\n'.join(self.statements) + "\n" if len(self.statements) else "") +
                "}")

class CodeGenerator(NodeVisitor):
    """
    Node visitor class that creates a code in C language.
    """
    def __init__(self):
        # List that holds all of the global variable/function declarations.
        self.globals = []

        # TODO: Complete if needed.

    _temporary_counter = 0
    def new_temporary(self, c_type):
        '''
        Declare a new temporary name
        '''
        CodeGenerator._temporary_counter += 1
        name = f'_t{CodeGenerator._temporary_counter}'
        self.function.locals.append(f'{c_type} {name};')
        return name

    _label_counter = 0        
    def new_label(self):
        '''
        Declare a new label name
        '''
        CodeGenerator._label_counter += 1
        return f'L{CodeGenerator._label_counter}'

    def declare_function(self, funcname, argdefns, rettype):
        # All emitted code must go into a function.        
        self.function = Function(funcname, argdefns, rettype)

        # Insert a reference in global namespace
        self.globals.append(self.function)

    def append(self, stmt):
        self.function.statements.append(stmt)

    def typeof(self, node):
        '''
        Returns the C type of a node
        '''
        uchuck_type = node.attrs['uchuck_type']
        if uchuck_type == IntType:
            return "int"
        elif uchuck_type == FloatType:
            return "double"
        elif uchuck_type == StringType:
            return "char*"
        else:
            raise RuntimeError(f'Unsupported type {uchuck_type}')

    def show(self, buf=sys.stdout):
        main = self.globals[0]
        gvar = self.globals[1:]
        _str  = "#include <stdio.h>\n"
        _str += "#include <stdlib.h>\n"
        _str += "#include <string.h>\n\n"
        _str += '\n'.join(gvar) + "\n\n" if len(gvar) else ""
        _str += str(main) + "\n"
        buf.write(_str)

    def visit_Program(self, node):
        # All emitted code must go into a function. For simplicity,
        # hardwire it to main. Declare a function called "main" that
        # takes no arguments and returns an integer.
        self.declare_function('main', [], 'int')
        # Visit all the statements
        for stmt in node.stmts:
            self.visit(stmt)
        # Generate the return statement
        self.append(f'return 0;')

    # You must implement visit_Nodename methods for all of the other
    # AST nodes.  In your code, you will need to make instructions
    # and append them to the list of generated instructions.
    
    # A few more example methods follow.
    # You must complete or change them if necessary.

    def visit_BinaryOp(self, node):
        # Visit the left and right expressions
        self.visit(node.left)
        lvalue  = node.left.attrs['gen_location']
        self.visit(node.right)
        rvalue = node.right.attrs['gen_location']
        # Allocate a new temporary for storing the result
        result = self.new_temporary(self.typeof(node))

        # TODO: Make the C code for the binary operation and
        # append to the list of generated instructions

        # Store the location of the result on the node
        node.attrs['gen_location'] = result

    def visit_ChuckOp(self, node):
        # Visit the expression (on the left)
        self.visit(node.expression)
        value = node.expression.attrs['gen_location']
        # Visit the location (on the right)
        self.visit(node.location)
        varname = node.location.attrs['gen_location']
        # Make the C code for the assignment operation and append to the list of generated instructions
        if self.typeof(node.location) == 'char*':
            self.append(f'{varname} = strcpy(realloc({varname}, strlen({value})+1), {value});')
        else:
            self.append(f'{varname} = {value};')
        # Store the location on the current node
        node.attrs['gen_location'] = varname
```

Todo código emitido deve ir para uma função. Para simplificar,  ele será vinculado à função `main`. A classe `Function` será usada para encapsular o código C dessa função. Além disso, outra coisa... tudo o que acontece aqui ocorre *após* a análise semântica. Portanto, você não precisa se concentrar em problemas relacionados a programas incorretos. Assuma que todos os programas estejam totalmente corretos com relação ao uso de tipos e nomes.

### Diretrizes

A seguir, é fornecido um conjunto de diretrizes que podem ser usadas para implementar a geração de código. Por favor, leia-as com atenção.

#### Programa

1. Programa (`visit_Program`)

- Declare uma função chamada "*main*" que não recebe argumentos e retorna um inteiro.
- Visite todos os comandos (`stmts`)
- Gere a instrução de retorno

#### Declarações / Tipo

1. Definição de variável (`visit_VarDecl`)

- Obtenha o nome (`name`) da variável
- Obtenha o tipo (`dtype`) da variável
- Gere o código C para a declaração da variável e insira na lista de declarações globais
- Armazene o nome da variável no nó atual (`attrs['gen_location']`)

2. Tipo (`visit_Type`)

- Não faça nada, apenas passe (`pass`)

#### Comandos

1. If (`visit_IfStatement`)

- Crie rótulos para os blocos necessários para gerar código C
- Visite a condição (`test`)
- Gere um desvio condicional para os blocos criados
- Vá para o primeiro bloco e gere o código C relacionado ao "então" (`consequence`)
- Gere um devio incondicional para o próximo bloco depois deste *if*
- Se houver um bloco "senão" (`alternative`), gere-o de maneira semelhante
- Gere código C para o rótulo do próximo bloco depois deste *if*

2. While (`visit_WhileStatement`)

- Crie rótulos para os blocos necessários para gerar código C
- Armazene os rótulos dos blocos no nó atual (`attrs['gen_blocks']`)
- Vá para o primeiro bloco e gere o código C relacionado a condição (`test`)
- Gere um desvio condicional para o corpo do laço
- Passe para o próximo bloco e gere o código C relacionado ao corpo do laço (`body`)
- Gere um desvio incondicional para o teste de condição do laço
- Gere código C para o rótulo do próximo bloco depois deste *while*

3. Break (`visit_BreakStatement`)

- Obtenha o nó do laço dentro do qual o comando *break* está (`attrs['loop']`)
- Obtenha os rótulos dos blocos criados para gerar este loop (`attrs['gen_blocks']`)
- Gerar um desvio incondicional para o próximo bloco após este laço

4. Continue (`visit_ContinueStatement`)

- Obtenha o nó do laço dentro do qual o comando *continue* está (`attrs['loop']`)
- Obtenha os rótulos dos blocos criados para gerar este loop (`attrs['gen_blocks']`)
- Gere um desvio incondicional para o bloco com o teste de condição deste laço

5. Expressão como comando (`visit_ExpressionAsStatement`)

- Se não estiver vazio, visite a expressão (`expression`)

6. Bloco (`visit_StmtList`)

- Basta visitar cada comando (`stmts`)

#### Expressões

1. Literal (`visit_Literal`)

- Converta o valor (`value`) para seu respectivo tipo (`type`)
- Armazene o valor convertido no nó atual (`attrs['gen_location']`)

2. Local (`visit_Location`)

- Armazene o nome da variável (`name`) no nó atual (`attrs['gen_location']`)

3. Operação binária (`visit_BinaryOp`)

- Visite as expressões esquerda e direita (`left`, `right`)
- Crie um novo temporário para armazenar o resultado
- Gere o código C para a operação binária e anexe-o à lista de instruções geradas
- Armazene o local do resultado no nó atual (`attrs['gen_location']`)

4. Operação unária (`visit_UnaryOp`)

- Comece visitando o operando da operação (`operand`)
- Crie um novo temporário para armazenar o resultado
- Gere o código C para a operação unária e anexe-o à lista de instruções geradas
- Armazene o local do resultado no nó atual (`attrs['gen_location']`)

5. Atribuição (`visit_ChuckOp`)

- Visite o valor (`expression`) (à esquerda)
- Visite o local (`location`) (à direita)
- Gere o código C para a operação atribuição e anexe-o à lista de instruções geradas
- Armazene o local no nó atual (`attrs['gen_location']`)

6. Impressão (`visit_PrintStatement`)

- Se for uma única expressão (não `ExprList`), crie uma lista com ela (`expression`), caso contrário, obtenha a lista de expressões
- Inicialize uma lista para armazenar os especificadores de formato dos elementos da lista
- Inicialize uma lista para armazenar os valores dos elementos da lista
- Itere sobre as expressões da lista
    - A cada iteração, visite uma expressão
    - Em seguida, de acordo com o seu tipo, insira seu especificador de formato e valor nas listas criadas no início
- Após a conclusão das iterações, gere uma instrução de impressão com especificadores de formato e valores das listas obtidas

7. Lista de expressões (`visit_ExprList`)

- Visite cada expressão (`exprs`)
- Armazene o local da última expressão no nó atual (`attrs['gen_location']`)

## Esboço do Gerador de Código

In [16]:
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UChuckLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1FXsbf3mp9rf-Cce-c9qk_8S43ppcPd4k?usp=sharing) e cole na célula abaixo.

In [32]:
class UChuckLexer(Lexer):
    """A lexer for the uChuck language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func

    # Reserved keywords
    keywords = {
        'int': "INT",
        'float': "FLOAT",
        'if': "IF",
        'else': "ELSE",
        'while': "WHILE",
        'break': "BREAK",
        'continue': "CONTINUE",
        'true': "TRUE",
        'false': "FALSE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "FLOAT_VAL",
        "INT_VAL",
        "STRING_LIT",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "PERCENT",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NEQ",
        "AND",
        "OR",
        "EXCLAMATION",
        # Assignment
        "CHUCK",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "L_HACK",
        "R_HACK",  # <<< >>>
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'\n+'
    ignore_comment = r'/\*(.|\n)*?\*/|//.*'

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    FLOAT_VAL = r'\d+\.\d+|\d+\.\d*|\.\d+'
    INT_VAL = r'\d+'
    STRING_LIT = r'"([^"\\]|\\.)*"'
    
    unterm_string = r'"(\\["\\]|[^"\\])*'
    unterm_comment = r'/\*.*'

    L_HACK = r'<<<'
    R_HACK = r'>>>'

    LE = r'<='
    GE = r'>='
    EQ = r'=='
    NEQ = r'!='
    AND = r'&&'
    OR = r'\|\|'
    CHUCK = r'=>'

    LT = r'<'
    GT = r'>'
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    PERCENT = r'%'
    EXCLAMATION = r'!'

    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r','
    SEMI = r';'

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    def ignore_comment(self, t):
        self.lineno += t.value.count("\n")

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_column(token)

    def unterm_comment(self, t):
        msg = "Unterminated comment"
        self._error(msg, t)

    def unterm_string(self, t):
        msg = "Unterminated string literal"
        self._error(msg, t)

    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

Copie o código das classes de nós AST que você escreveu no [terceiro projeto](https://colab.research.google.com/drive/1fRu6iseV-iWDrMtP7nR3yjame1RGEpnN?usp=sharing) e cole na célula abaixo.

In [33]:
import sys

def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord", "attrs",)

    def __init__(self, coord=None):
        self.coord = coord
        self.attrs = {}

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

class NodeVisitor:
    """ A base NodeVisitor class for visiting uchuck_ast nodes.
        Subclass it and define your own visit_XXX methods, where
        XXX is the class name you want to visit with these
        methods.
    """

    _method_cache = None

    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        for _, child in node.children():
            self.visit(child)

class BinaryOp(Node):

    __slots__ = ("op", "left", "right",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.left = left
        self.right = right

    def children(self):
        nodelist = []
        if self.left is not None:
            nodelist.append(('left', self.left))
        if self.right is not None:
            nodelist.append(('right', self.right))
        return tuple(nodelist)

    attr_names = ("op",)

class BreakStatement(Node):
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)

    def children(self):
        return ()

    attr_names = ()

class ChuckOp(Node):
    __slots__ = ("left", "right")

    def __init__(self, left, right, coord=None):
        super().__init__(coord)
        self.left = left
        self.right = right

    def children(self):
        return (('right', self.right), ('left', self.left))

    attr_names = ()

class ContinueStatement(Node):
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)

    def children(self):
        return ()

    attr_names = ()

class ExpressionAsStatement(Node):
    __slots__ = ("expression",)

    def __init__(self, expression, coord=None):
        super().__init__(coord)
        self.expression = expression

    def children(self):
        if self.expression is not None:
            return (('expression', self.expression),)
        return ()

    attr_names = ()

class ExprList(Node):
    __slots__ = ("exprs",)

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))
        return tuple(nodelist)

    attr_names = ()

class ID(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class IfStatement(Node):
    __slots__ = ("test", "consequence", "alternative")

    def __init__(self, test, consequence, alternative, coord=None):
        super().__init__(coord)
        self.test = test
        self.consequence = consequence
        self.alternative = alternative

    def children(self):
        nodelist = []
        if self.test is not None:
            nodelist.append(('test', self.test))
        if self.consequence is not None:
            nodelist.append(('consequence', self.consequence))
        if self.alternative is not None:
            nodelist.append(('alternative', self.alternative))
        return tuple(nodelist)

    attr_names = ()

class Literal(Node):

    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value",)

class Location(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class PrintStatement(Node):
    __slots__ = ("expression",)

    def __init__(self, expression, coord=None):
        super().__init__(coord)
        self.expression = expression

    def children(self):
        if self.expression is not None:
            return (('expression', self.expression),)
        return ()

    attr_names = ()

class Program(Node):

    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)

class StmtList(Node):
    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)

    attr_names = ()

class TypeNode(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class UnaryOp(Node):
    __slots__ = ("op", "operand")

    def __init__(self, op, operand, coord=None):
        super().__init__(coord)
        self.op = op
        self.operand = operand

    def children(self):
        nodelist = []
        if self.operand is not None:
            nodelist.append(('operand', self.operand))
        return tuple(nodelist)

    attr_names = ("op",)

class VarDecl(Node):
    __slots__ = ("dtype", "name")

    def __init__(self, dtype, name, coord=None):
        super().__init__(coord)
        self.dtype = dtype
        self.name = name

    def children(self):
        nodelist = []
        if self.dtype is not None:
            nodelist.append(('dtype', self.dtype))
        return tuple(nodelist)

    attr_names = ("name",)

class WhileStatement(Node):
    __slots__ = ("test", "body")

    def __init__(self, test, body, coord=None):
        super().__init__(coord)
        self.test = test
        self.body = body

    def children(self):
        nodelist = []
        if self.test is not None:
            nodelist.append(('test', self.test))
        if self.body is not None:
            nodelist.append(('body', self.body))
        return tuple(nodelist)

    attr_names = ()

Copie o código da classe `UChuckParser` que você escreveu no [terceiro projeto](https://colab.research.google.com/drive/1fRu6iseV-iWDrMtP7nR3yjame1RGEpnN?usp=sharing) e cole na célula abaixo.

In [34]:
class UChuckParser(Parser):
    """A parser for the uChuck language."""

    # Get the token list from the lexer (required)
    tokens = UChuckLexer.tokens

    precedence = (
        ('left', 'COMMA'),
        ('left', 'CHUCK'),
        ('left', 'OR'),
        ('left', 'AND'),
        ('left', 'EQ', 'NEQ'),
        ('left', 'LT', 'LE', 'GT', 'GE'),
        ('left', 'PLUS', 'MINUS'),
        ('left', 'TIMES', 'DIVIDE', 'PERCENT'),
        ('right', 'EXCLAMATION'),
    )

    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        self.lexer = UChuckLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    def _token_coord(self, p):
        return Coord(p.lineno, self.lexer.find_column(p))

    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= <statement_list> EOF
    @_('statement_list')
    def program(self, p):
        return Program(p.statement_list)

    # <statement_list> ::= { <statement> }+
    @_('statement statement_list')
    def statement_list(self, p):
        return [p.statement] + p.statement_list

    @_('statement')
    def statement_list(self, p):
        return [p.statement]

    # <statement> ::= <expression_statement>
    #               | <loop_statement>
    #               | <selection_statement>
    #               | <jump_statement>
    #               | <code_segment>
    @_('expression_statement',
       'loop_statement',
       'selection_statement',
       'jump_statement',
       'code_segment')
    def statement(self, p):
        return p[0]

    # <jump_statement> ::= "break" ";"
    #                    | "continue" ";"
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return BreakStatement(coord=self._token_coord(p))

    @_('CONTINUE SEMI')
    def jump_statement(self, p):
        return ContinueStatement(coord=self._token_coord(p))

    # <selection_statement> ::= "if" "(" <expression> ")" <statement> { "else" <statement> }?
    @_('IF LPAREN expression RPAREN statement ELSE statement')
    def selection_statement(self, p):
        return IfStatement(test=p.expression, consequence=p.statement0, alternative=p.statement1, coord=self._token_coord(p))

    @_('IF LPAREN expression RPAREN statement')
    def selection_statement(self, p):
        return IfStatement(test=p.expression, consequence=p.statement, alternative=None, coord=self._token_coord(p))

    # <loop_statement> ::= "while" "(" <expression> ")" <statement>
    @_('WHILE LPAREN expression RPAREN statement')
    def loop_statement(self, p):
        return WhileStatement(test=p.expression, body=p.statement, coord=self._token_coord(p))

    # <code_segment> ::= "{" { <statement_list> }? "}"
    @_('LBRACE statement_list RBRACE')
    def code_segment(self, p):
        return StmtList(stmts=p.statement_list, coord=self._token_coord(p))

    @_('LBRACE RBRACE')
    def code_segment(self, p):
        return StmtList(stmts=[], coord=self._token_coord(p))

    # <expression_statement> ::= { <expression> }? ";"
    @_('expression SEMI')
    def expression_statement(self, p):
        return ExpressionAsStatement(expression=p.expression, coord=None)

    @_('SEMI')
    def expression_statement(self, p):
        return ExpressionAsStatement(None, coord=None)

    # <expression> ::= <chuck_expression> { "," <chuck_expression> }*
    @_('expression COMMA expression')
    def expression(self, p):
        if isinstance(p.expression0, ExprList):
            expressions = p.expression0.exprs + [p.expression1]
            coord = p.expression0.coord
        else:
            expressions = [p.expression0, p.expression1]
            coord = p.expression0.coord
        
        return ExprList(exprs=expressions, coord=coord)

    @_('chuck_expression')
    def expression(self, p):
        return p.chuck_expression

    # <chuck_expression> ::= { <chuck_expression> "=>" }? <decl_expression>
    @_('chuck_expression CHUCK decl_expression')
    def chuck_expression(self, p):
        return ChuckOp(left=p.chuck_expression, right=p.decl_expression, coord=self._token_coord(p))
    
    @_('decl_expression')
    def chuck_expression(self, p):
        return p.decl_expression

    # <decl_expression> ::= <binary_expression>
    #                     | <type_decl> <identifier>
    @_('binary_expression')
    def decl_expression(self, p):
        return p.binary_expression

    @_('type_decl identifier')
    def decl_expression(self, p):
        return VarDecl(dtype=p.type_decl, name=ID(name=p.identifier.name, coord=p.identifier.coord), coord=None)

    # <type_decl> ::= "int"
    #               | "float"
    #               | <identifier>
    @_('INT')
    def type_decl(self, p):
        return TypeNode(name='int', coord=self._token_coord(p))

    @_('FLOAT')
    def type_decl(self, p):
        return TypeNode(name='float', coord=self._token_coord(p))

    @_('identifier')
    def type_decl(self, p):
        return TypeNode(name=p.identifier.name, coord=p.identifier.coord)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression> "+"  <binary_expression>
    #                       | <binary_expression> "-"  <binary_expression>
    #                       | <binary_expression> "*"  <binary_expression>
    #                       | <binary_expression> "/"  <binary_expression>
    #                       | <binary_expression> "%"  <binary_expression>
    #                       | <binary_expression> "<=" <binary_expression>
    #                       | <binary_expression> "<"  <binary_expression>
    #                       | <binary_expression> ">=" <binary_expression>
    #                       | <binary_expression> ">"  <binary_expression>
    #                       | <binary_expression> "==" <binary_expression>
    #                       | <binary_expression> "!=" <binary_expression>
    #                       | <binary_expression> "&&" <binary_expression>
    #                       | <binary_expression> "||" <binary_expression>
    @_('binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression',
       'binary_expression TIMES binary_expression',
       'binary_expression DIVIDE binary_expression',
       'binary_expression PERCENT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression LT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NEQ binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression',
       )
    def binary_expression(self, p):
        return BinaryOp(op=p[1], left=p.binary_expression0, right=p.binary_expression1, coord=p.binary_expression0.coord)
    
    @_('unary_expression')
    def binary_expression(self, p):
        return p.unary_expression

    # <unary_expression> ::= <primary_expression>
    #                      | <unary_operator> <unary_expression>
    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return UnaryOp(op=p.unary_operator, operand=p.unary_expression, coord=p.unary_expression.coord)

    @_('MINUS expression')
    def expression(self, p):
        return UnaryOp('-', p.expression, coord=self._token_coord(p.MINUS))

    @_('primary_expression')
    def unary_expression(self, p):
        return p.primary_expression

    # <unary_operator> ::= "+"
    #                    | "!"
    @_('PLUS')
    def unary_operator(self, p):
        return '+'
    
    @_('MINUS')
    def unary_operator(self, p):
        return '-'

    @_('EXCLAMATION')
    def unary_operator(self, p):
        return '!'

    # <primary_expression> ::= <literal>
    #                        | <location>
    #                        | "<<<" <expression> ">>>"
    #                        | "(" <expression> ")"
    @_('literal')
    def primary_expression(self, p):
        return p.literal

    @_('location')
    def primary_expression(self, p):
        return p.location

    @_('L_HACK expression R_HACK')
    def primary_expression(self, p):
        return PrintStatement(expression=p.expression, coord=self._token_coord(p))

    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return p.expression

    @_('INT_VAL')
    def literal(self, p):
        return Literal(type='int', value=p.INT_VAL, coord=self._token_coord(p))
    
    @_('FLOAT_VAL')
    def literal(self, p):
        return Literal(type='float', value=p.FLOAT_VAL, coord=self._token_coord(p))
    
    @_('STRING_LIT')
    def literal(self, p):
        return Literal(type='string', value=p.STRING_LIT, coord=self._token_coord(p))
    
    @_('TRUE')
    def literal(self, p):
        return Literal(type='int', value='1', coord=self._token_coord(p))

    @_('FALSE')
    def literal(self, p):
        return Literal(type='int', value='0', coord=self._token_coord(p))

    # <location> ::= <identifier>
    @_('identifier')
    def location(self, p):
        return Location(name=p.identifier.name, coord=p.identifier.coord)

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ID(name=p.ID, coord=Coord(p.lineno, self.lexer.find_column(p)))

In [35]:
class Type:
    """
    Class that represents a type in the language. Basic
    Types are declared as singleton instances of this type.
    """

    def __init__(self, name):
        self.typename = name

    def __repr__(self):
        return self.typename

class UChuckType(Type):

    def __init__(self, name, binary_ops=set(), unary_ops=set(), rel_ops=set()):
        super().__init__(name)
        self.binary_ops = binary_ops
        self.unary_ops = unary_ops
        self.rel_ops = rel_ops

# Create specific instances of basic types. You will need to add
# appropriate arguments depending on your definition of Type
IntType = UChuckType(
    "int",
    unary_ops  = {"-", "+", "!"},
    binary_ops = {"+", "-", "*", "/", "%"},
    rel_ops    = {"==", "!=", "<", ">", "<=", ">=", "&&", "||"},
)

FloatType = UChuckType(
    "float",
    unary_ops  = {"-", "+"},
    binary_ops = {"+", "-", "*", "/", "%"},
    rel_ops    = {"==", "!=", "<", ">", "<=", ">="},
)

StringType = UChuckType(
    "string",
    binary_ops = {"+"},
    rel_ops    = {"==", "!="},
)

In [36]:
class SymbolTable(dict):
    """ Class representing a symbol table. It should provide functionality
        for adding and looking up nodes associated with identifiers.
    """
    def __init__(self, parent=None):
        super().__init__()

    def add(self, name, value):
        self[name] = value

    def lookup(self, name):
        return self.get(name, None)

Copie o código da classe `Visitor` que você escreveu no [quarto projeto](https://colab.research.google.com/drive/1ulp9cLpEy2JKJ6JWUHjtDfqD4vBy0pg-?usp=sharing) e cole na célula abaixo.

In [52]:
class Visitor(NodeVisitor):
    '''
    Program visitor class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    '''
    def __init__(self):
        # Keep a reference to current symbol table
        self.symtab = None

        # Add built-in type names (int, float, string)
        self.typemap = {
            "int": IntType,
            "float": FloatType,
            "string": StringType,
        }
        
    def _assert_semantic(self, condition, msg_code, coord, name="", ltype="", rtype=""):
        """Check condition, if false print selected error message and exit"""
        error_msgs = {
             1: f"'{name}' is not defined",
             2: f"Cannot assign type '{rtype}' to type '{ltype}'",
             3: f"Binary operator '{name}' does not have matching LHS/RHS types",
             4: f"Binary operator '{name}' is not supported by type '{ltype}'",
             5: "Break/Continue statement must be inside a loop",
             6: f"The condition expression must be of type 'int', not type '{ltype}'",
             7: "Expression is not of basic type",
             8: f"Right-side operand is not a variable",
             9: f"Name '{name}' is already defined in this scope",
            10: f"Unary operator '{name}' is not supported by type '{ltype}'",
        }
        if not condition:
            msg = error_msgs.get(msg_code)
            print("SemanticError: %s %s" % (msg, coord), file=sys.stdout)
            sys.exit(1)
            
    def visit_BinaryOp(self, node):
        # Visit the left and right expression
        self.visit(node.left)
        ltype = node.left.attrs['uchuck_type']
        self.visit(node.right)
        rtype = node.right.attrs['uchuck_type']
        # TODO:
        # - Make sure left and right operands have the same type
        # - Make sure the operation is supported
        # - Assign the result type to current node
        self._assert_semantic(ltype == rtype, 3, node.coord, name=node.op)
        supported_ops = ltype.binary_ops.union(ltype.rel_ops)
        self._assert_semantic(node.op in supported_ops, 4, node.coord, name=node.op, ltype=ltype.typename)

        if node.op in ltype.rel_ops:
            node.attrs['uchuck_type'] = IntType
        else:
            node.attrs['uchuck_type'] = ltype

    def visit_BreakStatement(self, node):
        loops = self.symtab.lookup('loops')
        self._assert_semantic(len(loops) > 0, 5, node.coord)
        node.attrs['loop'] = loops[-1]

    def visit_ChuckOp(self, node):
        # Visita a expressão (à esquerda)
        self.visit(node.left)
        expr_type = node.left.attrs['uchuck_type']
        # Visita a localização (à direita)
        self.visit(node.right)
        loc_type = node.right.attrs['uchuck_type']
        
        # Garante que a localização é uma variável
        loc = node.right
        is_assignable = loc.__class__.__name__ in ('Location', 'VarDecl')
        self._assert_semantic(is_assignable, 8, coord=getattr(loc, 'coord', node.coord))
        
        # Checa se a atribuição é permitida
        self._assert_semantic(loc_type == expr_type, 2, node.coord, ltype=str(loc_type), rtype=str(expr_type))
        
        # Atribui o tipo da localização ao nó atual
        node.attrs['uchuck_type'] = loc_type

    def visit_ContinueStatement(self, node):
        loops = self.symtab.lookup('loops')
        self._assert_semantic(len(loops) > 0, 5, node.coord)
        node.attrs['loop'] = loops[-1]

    def visit_ExpressionAsStatement(self, node):
        if node.expression:
            self.visit(node.expression)

    def visit_ExprList(self, node):
        for expr in node.exprs:
            self.visit(expr)
        if node.exprs:
            node.attrs['uchuck_type'] = node.exprs[-1].attrs['uchuck_type']

    def visit_IfStatement(self, node):
        self.visit(node.test)
        test_type = node.test.attrs['uchuck_type']
        self._assert_semantic(test_type == IntType, 6, node.coord, ltype=str(test_type))
        self.visit(node.consequence)
        if node.alternative:
            self.visit(node.alternative)

    def visit_Literal(self, node):
        type_name = node.type
        if type_name == 'bool':
            type_name = 'int'
        node.attrs['uchuck_type'] = self.typemap.get(type_name)

    def visit_Location(self, node):
        decl_node = self.symtab.lookup(node.name)
        self._assert_semantic(decl_node is not None, 1, node.coord, name=node.name)
        node.attrs['defn'] = decl_node
        node.attrs['uchuck_type'] = decl_node.attrs['uchuck_type']

    def visit_PrintStatement(self, node):
        self.visit(node.expression)
        expr_type = node.expression.attrs.get('uchuck_type')
        self._assert_semantic(expr_type is not None, 7, node.coord)
        node.attrs['uchuck_type'] = expr_type

    def visit_Program(self, node):
        # Create a symbol table and assign it to current node
        node.attrs['symtab'] = SymbolTable()
        # Set the reference to current symbol table to the new symbol table
        self.symtab = node.attrs['symtab']
        # Create an empty list to bind loop nodes to nested break/continue statements and insert it into the symbol table
        self.symtab.add('loops', [])
        # Visit all the statements
        for stmt in node.stmts:
            self.visit(stmt)

    def visit_StmtList(self, node):
        if node.stmts:
            for stmt in node.stmts:
                self.visit(stmt)

    def visit_TypeNode(self, node):
        self._assert_semantic(node.name in self.typemap, 1, node.coord, name=node.name)
        node.attrs['uchuck_type'] = self.typemap[node.name]

    def visit_UnaryOp(self, node):
        self.visit(node.operand)
        operand_type = node.operand.attrs['uchuck_type']
        self._assert_semantic(node.op in operand_type.unary_ops, 10, node.coord, name=node.op, ltype=str(operand_type))
        node.attrs['uchuck_type'] = operand_type

    def visit_VarDecl(self, node):
        var_name = node.name.name
        self._assert_semantic(self.symtab.lookup(var_name) is None, 9, node.coord, name=var_name)
        
        self.visit(node.dtype)
        var_type = node.dtype.attrs['uchuck_type']
        
        node.attrs['uchuck_type'] = var_type
        self.symtab.add(var_name, node)

    def visit_WhileStatement(self, node):
        loops = self.symtab.lookup('loops')
        loops.append(node)
        
        self.visit(node.test)
        test_type = node.test.attrs['uchuck_type']
        self._assert_semantic(test_type == IntType, 6, node.coord, ltype=str(test_type))
        
        self.visit(node.body)
        loops.pop()

Complete o código abaixo, escrevendo um método `visit_XXX` para cada tipo de nó AST.

In [67]:
# Encapsulation of a C function
class Function:
    
    def __init__(self, funcname, argdefns, rettype):
        self.funcname = funcname
        self.argdefns = argdefns
        self.rettype = rettype
        self.locals = []
        self.statements = []
        
    def __str__(self):
        # Format local variable declarations
        local_decls = ""
        if self.locals:
            local_decls = '\n'.join([f'    {l}' for l in self.locals]) + '\n\n'

        # Format statements
        stmts = ""
        if self.statements:
            stmts_list = []
            for stmt in self.statements:
                # Add indentation to all lines except labels
                prefix = "    " if not stmt.endswith(':') else ""
                stmts_list.append(f'{prefix}{stmt}')
            stmts = '\n'.join(stmts_list) + '\n'

        return (f'{self.rettype} {self.funcname}(' + ', '.join(self.argdefns) + ") {\n" +
                local_decls +
                stmts +
                "}")

class CodeGenerator(NodeVisitor):
    """
    Node visitor class that creates a code in C language.
    """
    def __init__(self):
        # List that holds all of the global variable/function declarations.
        self.globals = []

        # Reference to the current function being generated
        self.function = None

        # Keep track of declared variables to avoid duplicates
        self.declared_vars = set()
        
    _temporary_counter = 0
    def new_temporary(self, c_type):
        '''
        Declare a new temporary name
        '''
        CodeGenerator._temporary_counter += 1
        name = f'_t{CodeGenerator._temporary_counter}'
        self.function.locals.append(f'{c_type} {name};')
        return name

    _label_counter = 0        
    def new_label(self):
        '''
        Declare a new label name
        '''
        CodeGenerator._label_counter += 1
        return f'L{CodeGenerator._label_counter}'

    def declare_function(self, funcname, argdefns, rettype):
        # All emitted code must go into a function.        
        self.function = Function(funcname, argdefns, rettype)
        # Insert a reference in global namespace
        self.globals.append(self.function)

    def append(self, stmt):
        self.function.statements.append(stmt)

    def typeof(self, node):
        '''
        Returns the C type of a node
        '''
        uchuck_type = node.attrs['uchuck_type']
        if uchuck_type == IntType:
            return "int"
        elif uchuck_type == FloatType:
            return "double"
        elif uchuck_type == StringType:
            return "char*"
        else:
            raise RuntimeError(f'Unsupported type {uchuck_type}')

    def show(self, buf=sys.stdout):
        gvar = [g for g in self.globals if isinstance(g, str)]
        main = next((g for g in self.globals if isinstance(g, Function) and g.funcname == 'main'), None)
        
        _str  = "#include <stdio.h>\n"
        _str += "#include <stdlib.h>\n"
        _str += "#include <string.h>\n\n"
        if gvar:
            _str += '\n'.join(gvar) + "\n\n"
        if main:
            _str += str(main) + "\n"
        buf.write(_str)

    def visit_Program(self, node):
        # All emitted code must go into a function. For simplicity, 
        # hardwire it to main. Declare a function called "main" that 
        # takes no arguments and returns an integer.
        self.declare_function('main', [], 'int')
        # Visit all the statements
        if node.stmts:
            for stmt in node.stmts:
                self.visit(stmt)
        # Generate the return statement
        self.append(f'return 0;')

    def visit_VarDecl(self, node):
        var_name = node.name.name
        if var_name not in self.declared_vars:
            self.declared_vars.add(var_name)
            c_type = self.typeof(node)
            declaration = f'{c_type} {var_name};'
            if c_type == 'char*':
                # Initialize string pointers to NULL
                declaration = f'{c_type} {var_name} = NULL;'
            self.globals.append(declaration)
        node.attrs['gen_location'] = var_name

    def visit_TypeNode(self, node):
        pass

    def visit_IfStatement(self, node):
        true_label = self.new_label()
        false_label = self.new_label()
        # If there's an else block, we need a final exit label
        exit_label = self.new_label() if node.alternative else false_label

        self.visit(node.test)
        test_var = node.test.attrs['gen_location']
        self.append(f'if ({test_var}) goto {true_label};')
        self.append(f'goto {false_label};')

        # Consequence (then) block
        self.append(f'{true_label}:')
        self.visit(node.consequence)
        if node.alternative:
            self.append(f'goto {exit_label};')

        # Alternative (else) block
        self.append(f'{false_label}:')
        if node.alternative:
            self.visit(node.alternative)
            self.append(f'{exit_label}:')

    def visit_WhileStatement(self, node):
        start_label = self.new_label()
        body_label = self.new_label()
        exit_label = self.new_label()

        # Store labels for break/continue to find
        node.attrs['gen_blocks'] = {'start': start_label, 'exit': exit_label}

        self.append(f'{start_label}:')
        self.visit(node.test)
        test_var = node.test.attrs['gen_location']
        self.append(f'if ({test_var}) goto {body_label};')
        self.append(f'goto {exit_label};')
        
        self.append(f'{body_label}:')
        self.visit(node.body)
        self.append(f'goto {start_label};')
        
        self.append(f'{exit_label}:')

    def visit_BreakStatement(self, node):
        loop_node = node.attrs['loop']
        exit_label = loop_node.attrs['gen_blocks']['exit']
        self.append(f'goto {exit_label};')

    def visit_ContinueStatement(self, node):
        loop_node = node.attrs['loop']
        start_label = loop_node.attrs['gen_blocks']['start']
        self.append(f'goto {start_label};')

    def visit_ExpressionAsStatement(self, node):
        if node.expression:
            self.visit(node.expression)

    def visit_StmtList(self, node):
        if node.stmts:
            for stmt in node.stmts:
                self.visit(stmt)

    def visit_Literal(self, node):
        # String literals need to be enclosed in quotes
        if self.typeof(node) == 'char*':
            node.attrs['gen_location'] = f'{node.value}'
        else:
            node.attrs['gen_location'] = node.value

    def visit_Location(self, node):
        node.attrs['gen_location'] = node.name
        
    def visit_BinaryOp(self, node):
        # Visit the left and right expressions 
        self.visit(node.left)
        lvalue  = node.left.attrs['gen_location']
        self.visit(node.right)
        rvalue = node.right.attrs['gen_location']
        # Allocate a new temporary for storing the result
        result = self.new_temporary(self.typeof(node))
        
        op = node.op
        # Special handling for string operations
        if self.typeof(node.left) == 'char*':
            if op == '+':
                # String concatenation
                len1 = self.new_temporary('int')
                len2 = self.new_temporary('int')
                total_len = self.new_temporary('int')
                self.append(f'{len1} = strlen({lvalue});')
                self.append(f'{len2} = strlen({rvalue});')
                self.append(f'{total_len} = {len1} + {len2};')
                self.append(f'{result} = malloc({total_len} + 1);')
                self.append(f'strcpy({result}, {lvalue});')
                self.append(f'strcat({result}, {rvalue});')
            elif op in ('==', '!='):
                # String comparison
                cmp_res = self.new_temporary('int')
                self.append(f'{cmp_res} = strcmp({lvalue}, {rvalue});')
                self.append(f'{result} = {cmp_res} {op} 0;')
            else:
                # Should be caught by semantic analysis, but as a fallback:
                raise RuntimeError(f"Unsupported binary operator '{op}' for strings")
        else:
            # Standard numeric/boolean operations
            self.append(f'{result} = {lvalue} {op} {rvalue};')

        # Store the location of the result on the node
        node.attrs['gen_location'] = result

    def visit_UnaryOp(self, node):
        self.visit(node.operand)
        value = node.operand.attrs['gen_location']
        result = self.new_temporary(self.typeof(node))
        self.append(f'{result} = {node.op}{value};')
        node.attrs['gen_location'] = result

    def visit_ChuckOp(self, node):
        # Visit expression on the left to get value
        self.visit(node.left)
        value = node.left.attrs['gen_location']
        # Visit location on the right to get variable name
        self.visit(node.right)
        varname = node.right.attrs['gen_location']
        
        # Make the C code for the assignment
        if self.typeof(node.right) == 'char*':
            # For strings, reallocate memory and copy
            self.append(f'{varname} = realloc({varname}, strlen({value}) + 1);')
            self.append(f'strcpy({varname}, {value});')
        else:
            self.append(f'{varname} = {value};')
            
        # The result of a chuck operation is the value assigned
        node.attrs['gen_location'] = varname

    def visit_PrintStatement(self, node):
        # An expression list might be a single expression
        expressions = node.expression.exprs if isinstance(node.expression, ExprList) else [node.expression]
        
        format_specifiers = []
        values = []
        
        for expr in expressions:
            self.visit(expr)
            values.append(expr.attrs['gen_location'])
            
            c_type = self.typeof(expr)
            if c_type == 'int':
                format_specifiers.append('%i')
            elif c_type == 'double':
                format_specifiers.append('%f')
            elif c_type == 'char*':
                format_specifiers.append('%s')

        format_string = "".join(format_specifiers) + "\\n"
        value_string = ", ".join(values)
        
        if value_string:
            self.append(f'printf("{format_string}", {value_string});')
        else:
            # Should not happen with valid uChuck syntax for print
            self.append(f'printf("{format_string}");')

    def visit_ExprList(self, node):
        if node.exprs:
            for expr in node.exprs:
                self.visit(expr)
            # The location of an expr list is the location of its last expression
            node.attrs['gen_location'] = node.exprs[-1].attrs['gen_location']

In [68]:
def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)

def main(args):
    parser = UChuckParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        ast = parser.parse(f.read())
        if ast is not None:
            sema = Visitor()
            sema.visit(ast)
            gen = CodeGenerator()
            gen.visit(ast)
            with open('out.c', 'w') as file:
                gen.show(file)
                print('Wrote: out.c')

## Teste
Para o desenvolvimento inicial, tente executar o gerador de código em um arquivo de entrada de exemplo, como:

```
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}
```

In [69]:
%%file test.uck
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}

Overwriting test.uck


In [70]:
main(["test.uck"])

Wrote: out.c


Isso irá criar um arquivo `out.c` com o programa em C, cujo conteúdo será semelhante ao texto mostrado abaixo.

```c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int n;
int value;

int main() {
int _t1;
int _t2;
int _t3;

n = 1;
value = 1;
L1:
_t1 = n < 10;
if (_t1) goto L2;
goto L3;
L2:
_t2 = value * n;
value = _t2;
printf("%i\n", value);
_t3 = n + 1;
n = _t3;
goto L1;
L3:
return 0;
}
```

In [71]:
%pycat out.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int n;
int value;

int main() {
    int _t1;
    int _t2;
    int _t3;

    n = 1;
    value = 1;
L1:
    _t1 = n < 10;
    if (_t1) goto L2;
    goto L3;
L2:
    _t2 = value * n;
    value = _t2;
    printf("%i\n", value);
    _t3 = n + 1;
    n = _t3;
    goto L1;
L3:
    return 0;
}


Para executá-lo, faça o seguinte:

In [72]:
!gcc out.c -o out.exe

In [73]:
!out.exe

1
2
6
24
120
720
5040
40320
362880
